In [ ]:
%pip install matplotlib numpy scipy

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy
import sys

sys.path.append(os.path.abspath(".."))

In [ ]:
accel_std = 0.03
velocity = np.cumsum(accel_std * np.random.normal(size=80))
position = np.cumsum(velocity)
position -= np.mean(position)
noise_std = 0.5
observations = position + noise_std * np.random.normal(size=len(position))


def plot_measures():
    plt.plot(position, label="Path")
    plt.scatter(np.arange(len(position)), observations, label="Noisy path observations")


plot_measures()
_ = plt.legend()

In [ ]:
import simprob
import simprob.kalman as kalman

dims = 2

init_state = kalman.MultivariateNormal(
    mean=np.zeros(dims),
    covar=1e12 * np.eye(dims),
)
kobs = [
    simprob.NoObservation()
    if np.isnan(x)
    else kalman.MultivariateNormalSubspace(
        dist=kalman.MultivariateNormal(
            mean=np.array([x]),
            covar=np.array([[noise_std**2]]),
        ),
        # The observation is only on the position component
        subspace=np.array([[1, 0]]),
    )
    for x in observations
]
steps = [
    simprob.Iteration(
        transition=kalman.KalmanTransition(
            transition_matrix=np.array([[1.0, 1], [0, 1]]),
            expected_state_change=kalman.MultivariateNormal(
                mean=np.zeros(dims),
                covar=np.array([[0, 0], [0, accel_std**2]]),
            ),
        ),
        observation=x,
    )
    for x in kobs[1:]
]

kalman_comb = list(simprob.simulate(kobs[0] & init_state, steps))

plot_measures()

means = np.array([x.mean[0] for x in kalman_comb])
stds = np.array([x.covar[0, 0] ** 0.5 for x in kalman_comb])
[means_plot] = plt.plot(means, label="Path estimation at each time with Kalman")
for p in np.linspace(0.05, 1, 7)[:-1]:
    s = scipy.stats.chi2(1).isf(p) ** 0.5
    for mult in [s, -s]:
        plt.plot(means + mult * stds, color=means_plot.get_color(), alpha=p)
plt.ylim(observations.min(), observations.max())
_ = plt.legend()